# **利用langchain chroma建立embeddings**

In [ ]:
!pip install openai

In [ ]:
pip install -qU "langchain-chroma>=0.1.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.5/612.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.3 MB/s eta 0:00:0

In [ ]:
pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

# 掛載 Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
from langchain_chroma import Chroma

persist_directory = "/content/drive/MyDrive/vector_store"

vector_store = Chroma(
    collection_name="vector_store_jobs_document_short",
    embedding_function=embeddings,
    persist_directory= persist_directory ,  # Where to save data locally, remove if not necessary
)

In [ ]:
import re
from langchain_core.documents import Document
import json

with open('job_description_short.txt', 'r', encoding="utf-8") as file:
    lines = file.readlines()
    documents = []
    id_list = []
    for job in lines:
      if job.strip():  #確保行不為空
          job = job.replace("'", '"')
          job_no = json.loads(job)['job_no']
          #id_list.append(job_no)
          job = job.strip()
          job = job.replace("\\n", " ")
          job = job[:2000]
          documents=Document(
          page_content= job,
          metadata={"source": "recommendation"},
          id = job_no)
          vector_store.add_documents([documents], ids = [job_no])


In [ ]:
vector_store.get(include=["embeddings", "documents"])

{'ids': ['8i1vf', '8anmk', '8e5se', '8iejf', '714cp', '8fhgl', '8ebty'],
 'embeddings': array([[-2.59150099e-02, -1.26332296e-02, -3.32551170e-03, ...,
         -8.82974919e-03, -8.45142733e-03, -1.81729347e-02],
        [-3.35998759e-02,  6.34850236e-03,  1.66794602e-02, ...,
          1.52939674e-03, -1.84866246e-02,  1.16963708e-03],
        [-2.28178240e-02, -4.05800380e-02, -1.08056853e-03, ...,
         -1.00030741e-02, -4.48786579e-02, -9.66513250e-03],
        ...,
        [-1.31101422e-02, -6.67547015e-03,  1.02941366e-02, ...,
         -2.14712024e-02, -3.06884348e-02,  1.69562292e-03],
        [-3.00195832e-02, -7.44784847e-05,  2.05269046e-02, ...,
          1.04460039e-03, -2.38398761e-02, -9.72762937e-04],
        [-3.25273499e-02, -4.71238280e-03,  1.09762801e-02, ...,
         -1.58417728e-02, -3.25273499e-02, -2.35789269e-03]]),
 'documents': ['{"job_no": "8i1vf", "job_company": "台灣英特艾倫人力資源有限公司", "job_company_link": "https://www.104.com.tw/company/1a2x6bj0ov?jobsource=

In [ ]:
results = vector_store.similarity_search_with_score(
    "與房地產有關的職缺?", k=2 #, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.353143] {"job_no": "8e5se", "job_company": "信義房屋股份有限公司", "job_company_link": "https://www.104.com.tw/company/a9pjy3k?jobsource=index_job_r", "job_name": "【PropTech數位人才招募】AI研究員/數據科學家│AI數據應用發展 全職", "job_salary": "（經常性薪資達 4 萬元或以上）待遇面議", "job_description": "【關於PropTech數位人才招募計劃】 信義企業集團的各項產品與計畫正融入大數據、機器學習與人工智慧等應用，我們的數位團隊也持續致力於「房地產科技創新」與「數位居住生活」兩大面向，以創新智能工具，為使用者提供超乎想像的服務體驗。 我們希望讓居住在這片土地上的人們，在「不動產交易買賣」與「居住生活日常」有更好的保障與體驗，也期待你與我們一起，加速這個改變。 這裡匯聚了40年的集團產業資源，以及多元化的人才；接下來，想邀請你加入，與我們一起創造超乎想像的體驗，成就即將到來的未來！  【職務說明】#團隊擴編 一起成為加速AI應用的推動者 1. 研究最新AI演算法與應用，利用AI技術支持概念實現 2. 協助建立並優化AI模型，提升預測精確度 3. 與跨部門團隊合作，將AI技術整合到實際業務場景中", "job_location": "台北市信義區信義路五段100號(110)", "job_requirements": "1. 3年以上人工智慧/機器學習/深度學習，並有實際專案應用經驗 2. 對數據分析與演算法解決商業問題有高度興趣 3. 熟悉AI框架如TensorFlow, PyTorch等，熟悉程式語言如Python, SQL 4. 具備獨立研究與快速學習新技術的能力 5. 關注AI領域最新研究動向，並能將其應用到實際場景中 6. 邏輯思考能力強，且具良好溝通能力 7. 有耐心，思考清晰有條理、能換位溝通最重要  加分條件： 有實戰經驗：對商業議題能釐清並評估需求、定義假說並實驗設計、驗證優化後完成觀察指標設計", "job_welfare": "對信義房屋而言，人才不是財產，而是夥伴。我們深知人才影響企業的榮枯，要提供好的服務體驗，首要，就要善待人才。

# **自Google Drive load vectorstore進來**



In [ ]:
from google.colab import drive

# 掛載 Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
persist_directory = "/content/drive/MyDrive/vector_store"

In [ ]:
from langchain_chroma import Chroma
loaded_db= Chroma(collection_name="vector_store_jobs_document_short",persist_directory=persist_directory, embedding_function=embeddings)

In [ ]:
loaded_db.get(include=["embeddings", "documents"])

{'ids': ['8i1vf', '8anmk', '8e5se', '8iejf', '714cp', '8fhgl', '8ebty'],
 'embeddings': array([[-2.59150099e-02, -1.26332296e-02, -3.32551170e-03, ...,
         -8.82974919e-03, -8.45142733e-03, -1.81729347e-02],
        [-3.35998759e-02,  6.34850236e-03,  1.66794602e-02, ...,
          1.52939674e-03, -1.84866246e-02,  1.16963708e-03],
        [-2.28178240e-02, -4.05800380e-02, -1.08056853e-03, ...,
         -1.00030741e-02, -4.48786579e-02, -9.66513250e-03],
        ...,
        [-1.31101422e-02, -6.67547015e-03,  1.02941366e-02, ...,
         -2.14712024e-02, -3.06884348e-02,  1.69562292e-03],
        [-3.00195832e-02, -7.44784847e-05,  2.05269046e-02, ...,
          1.04460039e-03, -2.38398761e-02, -9.72762937e-04],
        [-3.25273499e-02, -4.71238280e-03,  1.09762801e-02, ...,
         -1.58417728e-02, -3.25273499e-02, -2.35789269e-03]]),
 'documents': ['{"job_no": "8i1vf", "job_company": "台灣英特艾倫人力資源有限公司", "job_company_link": "https://www.104.com.tw/company/1a2x6bj0ov?jobsource=

In [ ]:
#刪除整個vectorstore
#vector_store._client.delete_collection(vector_store._collection.name)

In [18]:
results = loaded_db.similarity_search_with_relevance_scores(
    "屬於科技業的職缺?", k=2 #, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.753500] {"job_no": "8e5se", "job_company": "信義房屋股份有限公司", "job_company_link": "https://www.104.com.tw/company/a9pjy3k?jobsource=index_job_r", "job_name": "【PropTech數位人才招募】AI研究員/數據科學家│AI數據應用發展 全職", "job_salary": "（經常性薪資達 4 萬元或以上）待遇面議", "job_description": "【關於PropTech數位人才招募計劃】 信義企業集團的各項產品與計畫正融入大數據、機器學習與人工智慧等應用，我們的數位團隊也持續致力於「房地產科技創新」與「數位居住生活」兩大面向，以創新智能工具，為使用者提供超乎想像的服務體驗。 我們希望讓居住在這片土地上的人們，在「不動產交易買賣」與「居住生活日常」有更好的保障與體驗，也期待你與我們一起，加速這個改變。 這裡匯聚了40年的集團產業資源，以及多元化的人才；接下來，想邀請你加入，與我們一起創造超乎想像的體驗，成就即將到來的未來！  【職務說明】#團隊擴編 一起成為加速AI應用的推動者 1. 研究最新AI演算法與應用，利用AI技術支持概念實現 2. 協助建立並優化AI模型，提升預測精確度 3. 與跨部門團隊合作，將AI技術整合到實際業務場景中", "job_location": "台北市信義區信義路五段100號(110)", "job_requirements": "1. 3年以上人工智慧/機器學習/深度學習，並有實際專案應用經驗 2. 對數據分析與演算法解決商業問題有高度興趣 3. 熟悉AI框架如TensorFlow, PyTorch等，熟悉程式語言如Python, SQL 4. 具備獨立研究與快速學習新技術的能力 5. 關注AI領域最新研究動向，並能將其應用到實際場景中 6. 邏輯思考能力強，且具良好溝通能力 7. 有耐心，思考清晰有條理、能換位溝通最重要  加分條件： 有實戰經驗：對商業議題能釐清並評估需求、定義假說並實驗設計、驗證優化後完成觀察指標設計", "job_welfare": "對信義房屋而言，人才不是財產，而是夥伴。我們深知人才影響企業的榮枯，要提供好的服務體驗，首要，就要善待人才。

# **建立QA Assistant**

In [21]:
retriever = loaded_db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [24]:
retrieved_docs = retriever.invoke("房地產相關職缺")



In [28]:
print(retrieved_docs[0].page_content)



{"job_no": "8e5se", "job_company": "信義房屋股份有限公司", "job_company_link": "https://www.104.com.tw/company/a9pjy3k?jobsource=index_job_r", "job_name": "【PropTech數位人才招募】AI研究員/數據科學家│AI數據應用發展 全職", "job_salary": "（經常性薪資達 4 萬元或以上）待遇面議", "job_description": "【關於PropTech數位人才招募計劃】 信義企業集團的各項產品與計畫正融入大數據、機器學習與人工智慧等應用，我們的數位團隊也持續致力於「房地產科技創新」與「數位居住生活」兩大面向，以創新智能工具，為使用者提供超乎想像的服務體驗。 我們希望讓居住在這片土地上的人們，在「不動產交易買賣」與「居住生活日常」有更好的保障與體驗，也期待你與我們一起，加速這個改變。 這裡匯聚了40年的集團產業資源，以及多元化的人才；接下來，想邀請你加入，與我們一起創造超乎想像的體驗，成就即將到來的未來！  【職務說明】#團隊擴編 一起成為加速AI應用的推動者 1. 研究最新AI演算法與應用，利用AI技術支持概念實現 2. 協助建立並優化AI模型，提升預測精確度 3. 與跨部門團隊合作，將AI技術整合到實際業務場景中", "job_location": "台北市信義區信義路五段100號(110)", "job_requirements": "1. 3年以上人工智慧/機器學習/深度學習，並有實際專案應用經驗 2. 對數據分析與演算法解決商業問題有高度興趣 3. 熟悉AI框架如TensorFlow, PyTorch等，熟悉程式語言如Python, SQL 4. 具備獨立研究與快速學習新技術的能力 5. 關注AI領域最新研究動向，並能將其應用到實際場景中 6. 邏輯思考能力強，且具良好溝通能力 7. 有耐心，思考清晰有條理、能換位溝通最重要  加分條件： 有實戰經驗：對商業議題能釐清並評估需求、定義假說並實驗設計、驗證優化後完成觀察指標設計", "job_welfare": "對信義房屋而言，人才不是財產，而是夥伴。我們深知人才影響企業的榮枯，要提供好的服務體驗，首要，就要善待人才。  ※身心健康有保障※ 1.【員工

In [29]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [30]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [32]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [33]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [35]:
for chunk in rag_chain.stream("請問與房地產最相關的職缺名稱、公司名稱及職務內容為何，請精簡回答"):
    print(chunk, end="", flush=True)

與房地產最相關的職缺為「AI研究員/數據科學家」，公司名稱為信義房屋股份有限公司。職務內容包括研究AI演算法與應用，協助建立優化AI模型，並將AI技術整合到業務中。薪資待遇為經常性薪資達4萬元或以上，具體待遇面議。

# **利用openai建立embedding**


In [ ]:
from dotenv import load_dotenv
import os
import openai

# Load the .env file
load_dotenv()

# Access environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
print(openai_api_key)

openai.api_key = openai_api_key  # 請替換為您的 OpenAI API 金鑰

In [ ]:
import openai
openai.api_key = openai_api_key  # 請替換為您的 OpenAI API 金鑰

In [ ]:
count= 0
with open('job_description_short.txt', 'r') as file:
    for line in file:
      count+=1
      print(line.strip())
print(count)

In [ ]:
import json
from openai import OpenAI

client = OpenAI()

def get_openai_embedding(text):
    response = client.embeddings.create(input=text, model="text-embedding-ada-002")
    return response.data[0].embedding

text_embedding = []

with open("job_description_short.txt", "r") as jobs :
  for job in jobs:
        if job.strip():  #確保行不為空
          job = job.replace("\n", " ")
          job = job[:5000]
          embedding = get_openai_embedding(job)
          print(job)
          text_embedding.append({"text": job, "embedding": embedding})

In [ ]:
# 將結果寫入到新的 JSON 文件中
with open('job_text_embedding_short.txt', 'w', encoding="utf-8" ) as output_file:
      output_file.write(json.dumps(text_embedding,ensure_ascii=False))

In [ ]:
with open('job_text_embedding_short.txt', 'r') as file:
   jobs = json.load(file)

for job in jobs:
  print(job['text'])
  print(job['embedding'])

In [ ]:
from chromadb import Client
from chromadb.config import Settings

# 初始化 Chroma
client = Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory="/content/drive/MyDrive/chroma_db"))

# 建立 collection
collection = client.create_collection("my_collection")

# 插入資料到 Chroma Vectorstore
for idx, embedding in enumerate(embeddings):
    collection.add(
        documents=[texts[idx]],  # 原始文本
        embeddings=[embedding],  # 向量
        metadatas=[{"id": idx}]  # 可以加入 metadata（如ID或其他資訊）
    )